This is a notebook mainly used for testing implementations and experimenting with new features.

Imported libraries

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
# import torch
# import torch.nn as nn

# import torch.optim as optim

# Source: https://stackoverflow.com/questions/48873483/python-example-for-wacom-tablets
from PyQt5 import (
    QtCore,
    QtGui,
    QtWidgets
)
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

# Source: https://stackoverflow.com/questions/54003194/how-can-i-access-tablet-pen-data-via-python
# import pyglet

# import evdev


Raw Input-Data extraction

NO SUCCESS

In [2]:
# devices = [evdev.InputDevice(path) for path in evdev.list_devices()]

# for device in devices:
#     print(device.path, device.name, device.phys)

In [3]:
# device = evdev.InputDevice('/dev/input/mouse3')
# print(device)

In [4]:
# devices = [evdev.InputDevice(path) for path in evdev.list_devices()]

# for device in devices:
#     print(device.path, device.name, device.phys)

Visualising the input data of the tablet, but not in optimal way.

Source: https://stackoverflow.com/questions/48873483/python-example-for-wacom-tablets

In [5]:
class TabletSampleWindow(QWidget):
    def __init__(self, parent=None):
        super(TabletSampleWindow, self).__init__(parent)
        # super().__init__()
        self.f = open("strokes/test.txt", "w")
        self.stroke_counter = 0

        self.pen_is_down = False
        self.pen_x = 0
        self.pen_y = 0
        self.pen_pressure = 0
        self.text = ""
        # Resizing the sample window to full desktop size:
        frame_rect = app.desktop().frameGeometry()
        width, height = frame_rect.width(), frame_rect.height()
        self.resize(width, height)
        self.move(-9, 0)
        self.setWindowTitle("Sample Tablet Event Handling")

    def tabletEvent(self, tabletEvent):
        self.pen_x = tabletEvent.pos()
        self.pen_y = tabletEvent.pos()
        self.pen_pressure = int(tabletEvent.pressure() * 100)
        if tabletEvent.type() == QTabletEvent.TabletPress:
            self.pen_is_down = True
            self.text = "TabletPress event"
        elif tabletEvent.type() == QTabletEvent.TabletMove:
            self.pen_is_down = True
            self.text = "TabletMove event"
        elif tabletEvent.type() == QTabletEvent.TabletRelease:
            self.pen_is_down = False
            self.text = "TabletRelease event"
        self.text += " at x={0}, y={1}, pressure={2}%,".format(self.pen_x, self.pen_y, self.pen_pressure)
        
        if self.pen_is_down:
            # self.f = open(f"strokes/stroke_{self.stroke_counter}.txt", "w")

            self.text += " Pen is down."
        else:
            self.text += " Pen is up."
            # self.stroke_counter += 1
            # self.f.close()
            # self.f = open(f"strokes/stroke_{self.stroke_counter}.txt", "w")

        self.f.write(f"[{self.pen_x},{self.pen_y},{self.pen_pressure}]\n")
        
        tabletEvent.accept()
        self.update()

    def paintEvent(self, event):
        text = self.text
        i = text.find("\n\n")
        if i >= 0:
            text = text.left(i)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.TextAntialiasing)
        painter.drawText(self.rect(), Qt.AlignTop | Qt.AlignLeft , text)


In [6]:
class InputWidget(QWidget):
    """ Input widget. """
    def __init__(self, width, height, which_type):
        """ Initializer. """
        super().__init__()
        # self.label = QLabel()
        # self.label.setText("Input field")
        # self.label.setFixedSize(100, 20)
        # self.label.setAlignment(
        #         QtCore.Qt.AlignHCenter | QtCore.Qt.AlignVCenter
        # )

        self.widget = QLabel()
        self.widget.setAlignment(
                QtCore.Qt.AlignHCenter | QtCore.Qt.AlignVCenter
        )
        self.setCursor(QtCore.Qt.CrossCursor)

        canvas = QPixmap(width, height)
        colour = QtGui.QColor(255, 255, 255, 220)
        canvas.fill(colour) # Fill canvas with color (default is white).
        self.widget.setPixmap(canvas)
        # self.setCentralWidget(self.widget)
        self.widget.show()

        # layout = QVBoxLayout()
        # layout.addWidget(self.label, alignment=QtCore.Qt.AlignCenter)
        # layout.addWidget(self.widget, alignment=QtCore.Qt.AlignCenter)
        # self.setLayout(layout)

        self.initialize_offsets()

        self.last_x, self.last_y = None, None
        self.current_stroke = []
        self.history = []

        self.setStyleSheet("background-image: url(alpha.png);")


    def initialize_offsets(self):
        """
        Keep track of the offset of the drawing canvas in the widget and the
        cursor offset while drawing.
        """
        self.canvas_offset_left = (self.widget.width() -
                self.widget.pixmap().width())/2
        self.canvas_offset_top = (self.widget.height() -
                self.widget.pixmap().height())/2
        self.cursor_offset_top = 0
        self.cursor_offset_left = 0

    def getHistory(self):
        """ Returns current input. """
        return self.history

    def getCurrentStroke(self):
        """ Returns the values stored for the current stroke. """
        return self.current_stroke

    def getLastX(self):
        """ Returns last x coordinate. """
        return self.last_x

    def getLastY(self):
        """ Returns last y coordinate. """
        return self.last_y

    def clearCanvas(self):
        """ Clears drawing canvas. """
        self.widget.pixmap().fill()
        self.history = []
        self.update()

    def tabletEvent(self, event):
        """ Handles tablet events. """
        current_x = int(event.x() - self.canvas_offset_left -
                self.cursor_offset_left)
        current_y = int(event.y() - self.canvas_offset_top -
                self.cursor_offset_top)

        if self.last_x is None: # First event.
            self.last_x = current_x
            self.last_y = current_y
            return # Ignore the first time.

        painter = QPainter(self.widget.pixmap())
        painter.drawLine(self.last_x, self.last_y, current_x, current_y)
        painter.end()
        self.update()

        # Update the origin for next time.
        self.last_x = current_x
        self.last_y = current_y
        self.current_stroke.append(
                [
                    self.last_x,
                    self.last_y,
                    QtCore.QDateTime.currentMSecsSinceEpoch()
                ]
        )

    def mouseReleaseEvent(self, event):
        """
        If the mouse is released that means the end of the current stroke.
        The stroke is added to the history.
        """
        if len(self.current_stroke) > 0:
            self.history.append(self.current_stroke)

        self.current_stroke = []
        self.last_x = None
        self.last_y = None

    def resizeEvent(self, event):
        """ Handles resizing of the canvas. """
        self.canvas_offset_left =(self.widget.width() -
                self.widget.pixmap().width())/2
        self.canvas_offset_top =(self.widget.height() -
                self.widget.pixmap().height())/2

In [7]:
app = QApplication(sys.argv)
mainform = InputWidget(600, 400, "alpha")
input = []
mainform.show()
app.exec_()

0